# Console

In this notebook, we debug our DAGs without running then through Apache Airflow.

In [13]:
import sys
sys.path.append('airflow/dags/lib')
import emrspark_lib as emrs
import configparser
import time

import logging
import os
import json

logger = logging.getLogger()
logger.setLevel(logging.INFO)

config = configparser.ConfigParser()
config.read('airflow/config.cfg')

CLUSTER_NAME = config['AWS']['CLUSTER_NAME']
VPC_ID = config['AWS']['VPC_ID']
SUBNET_ID = config['AWS']['SUBNET_ID']

if config['App']['STOCKS'] == '':
    STOCKS = []
else:
    STOCKS = json.loads(config.get('App', 'STOCKS').replace("'", '"'))

ec2, emr, iam = emrs.get_boto_clients(config['AWS']['REGION_NAME'], config=config)

if VPC_ID == '':
    VPC_ID = emrs.get_first_available_vpc(ec2)

if SUBNET_ID == '':
    SUBNET_ID = emrs.get_first_available_subnet(ec2, VPC_ID)

INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): ec2.us-east-1.amazonaws.com


In [ ]:
master_sg_id = emrs.create_security_group(ec2, '{}SG'.format(CLUSTER_NAME),
    'Master SG for {}'.format(CLUSTER_NAME), VPC_ID)
slave_sg_id = emrs.create_security_group(ec2, '{}SlaveSG'.format(CLUSTER_NAME),
    'Slave SG for {}'.format(CLUSTER_NAME), VPC_ID)

keypair = emrs.recreate_key_pair(ec2, '{}_pem'.format(CLUSTER_NAME))

emrs.recreate_default_roles(iam)

cluster_id = emrs.create_emr_cluster(emr, CLUSTER_NAME,
                master_sg_id,
                slave_sg_id,
                keypair['KeyName'], SUBNET_ID,
                release_label='emr-5.28.1')
cluster_dns = emrs.get_cluster_dns(emr, cluster_id)

## Pull Stock Info

In [9]:
args = {
    'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
    'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
    'START_DATE': config['App']['START_DATE'],
    'URL_NASDAQ': 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download',
    'URL_NYSE': 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nyse&render=download',
    'DB_HOST': 's3a://short-interest-effect',
    'TABLE_STOCK_INFO_NASDAQ': '/data/raw/stock_info_nasdaq',
    'TABLE_STOCK_INFO_NYSE': '/data/raw/stock_info_nyse',
}

In [12]:
emrs.kill_all_inactive_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/pull_stock_info.py',
        args=args,
        commonpath='airflow/dags/etl/common.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")

INFO:root:Killing all inactive spark sessions
INFO:root:Killed idle spark session id 0
INFO:root:Sent spark session creation command to http://ec2-54-92-211-100.compute-1.amazonaws.com:8998/sessions
INFO:root:Response headers: {'Date': 'Thu, 16 Jan 2020 20:49:27 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Encoding': 'gzip', 'Location': '/sessions/1', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.3.24.v20180605)'}
INFO:root:{'id': 1, 'name': None, 'appId': None, 'owner': None, 'proxyUser': None, 'state': 'starting', 'kind': 'pyspark', 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None}, 'log': ['stdout: ', '\nstderr: ', '\nYARN Diagnostics: ']}
INFO:root:Session headers: {'Date': 'Thu, 16 Jan 2020 20:49:27 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Encoding': 'gzip', 'Location': '/sessions/1', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.3.24.v20180605)'}


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Pull Short Interests

In [ ]:
args_si = {
    'START_DATE': config['App']['START_DATE'],
    'QUANDL_API_KEY': config['Quandl']['API_KEY'],
    'YESTERDAY_DATE': '2020-12-10',
    'LIMIT': config['App']['STOCK_LIMITS'],
    'STOCKS': STOCKS,
    'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
    'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
    'DB_HOST': config['App']['DB_HOST'],
    'TABLE_STOCK_INFO_NASDAQ': config['App']['TABLE_STOCK_INFO_NASDAQ'],
    'TABLE_STOCK_INFO_NYSE': config['App']['TABLE_STOCK_INFO_NYSE'],
    'TABLE_SHORT_INTERESTS_NASDAQ': config['App']['TABLE_SHORT_INTERESTS_NASDAQ'],
    'TABLE_SHORT_INTERESTS_NYSE': config['App']['TABLE_SHORT_INTERESTS_NYSE'],
}

In [ ]:
emrs.kill_all_inactive_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/pull_short_interests.py',
        args=args_si,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")

## Pull Prices

In [ ]:
args_p = {
    'START_DATE': config['App']['START_DATE'],
    'QUANDL_API_KEY': config['Quandl']['API_KEY'],
    'YESTERDAY_DATE': '2020-12-10',
    'LIMIT': config['App']['STOCK_LIMITS'],
    'STOCKS': STOCKS,
    'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
    'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
    'DB_HOST': config['App']['DB_HOST'],
    'TABLE_STOCK_INFO_NASDAQ': config['App']['TABLE_STOCK_INFO_NASDAQ'],
    'TABLE_STOCK_INFO_NYSE': config['App']['TABLE_STOCK_INFO_NYSE'],
    'TABLE_STOCK_PRICES': config['App']['TABLE_STOCK_PRICES'],
    'URL': "http://app.quotemedia.com/quotetools/getHistoryDownload.csv?&webmasterId=501&startDay={sd}&startMonth={sm}&startYear={sy}&endDay={ed}&endMonth={em}&endYear={ey}&isRanged=true&symbol={sym}",
}

In [ ]:
emrs.kill_all_inactive_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/pull_prices.py',
        args=args_p,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")

## Quality-check

In [ ]:
args_q = {
            'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
            'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
            'YESTERDAY_DATE': '2020-12-10',
            'STOCKS': STOCKS,
            'DB_HOST': config['App']['DB_HOST'],
            'TABLE_STOCK_INFO_NASDAQ': config['App']['TABLE_STOCK_INFO_NASDAQ'],
            'TABLE_STOCK_INFO_NYSE': config['App']['TABLE_STOCK_INFO_NYSE'],
            'TABLE_STOCK_PRICES': config['App']['TABLE_STOCK_PRICES'],
        }

In [ ]:
emrs.kill_all_inactive_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/pull_prices_quality.py',
        args=args_q,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")

## Combine Data

In [15]:
args_c = {
            'YESTERDAY_DATE': '2020-12-10',
            'AWS_ACCESS_KEY_ID': config['AWS']['AWS_ACCESS_KEY_ID'],
            'AWS_SECRET_ACCESS_KEY': config['AWS']['AWS_SECRET_ACCESS_KEY'],
            'DB_HOST': config['App']['DB_HOST'],
            'TABLE_STOCK_PRICES': config['App']['TABLE_STOCK_PRICES'],
            'TABLE_SHORT_INTERESTS_NASDAQ': config['App']['TABLE_SHORT_INTERESTS_NASDAQ'],
            'TABLE_SHORT_INTERESTS_NYSE': config['App']['TABLE_SHORT_INTERESTS_NYSE'],
            'TABLE_SHORT_ANALYSIS': config['App']['TABLE_SHORT_ANALYSIS'],
        }

In [16]:
logger.setLevel(logging.INFO)
emrs.kill_all_inactive_spark_sessions(cluster_dns)
session_headers = emrs.create_spark_session(cluster_dns)
emrs.wait_for_spark(cluster_dns, session_headers)
job_response_headers = emrs.submit_spark_job_from_file(
        cluster_dns, session_headers,
        'airflow/dags/etl/combine.py',
        args=args_q,
        commonpath='airflow/dags/etl/common.py',
        helperspath='airflow/dags/etl/helpers.py'
)
final_status, logs = emrs.track_spark_job(cluster_dns, job_response_headers)
emrs.kill_spark_session(cluster_dns, session_headers)
for line in logs:
    logging.info(line)
    if '(FAIL)' in str(line):
        logging.error(line)
        raise Exception("ETL process fails.")

INFO:root:Killing all inactive spark sessions
INFO:root:Sent spark session creation command to http://ec2-54-167-10-36.compute-1.amazonaws.com:8998/sessions
INFO:root:Response headers: {'Date': 'Thu, 16 Jan 2020 21:29:57 GMT', 'Content-Type': 'application/json', 'Content-Encoding': 'gzip', 'Location': '/sessions/2', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.2.16.v20160414)'}
INFO:root:{'id': 2, 'appId': None, 'owner': None, 'proxyUser': None, 'state': 'starting', 'kind': 'pyspark', 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None}, 'log': ['stdout: ', '\nstderr: ', '\nYARN Diagnostics: ']}
INFO:root:Session headers: {'Date': 'Thu, 16 Jan 2020 21:29:57 GMT', 'Content-Type': 'application/json', 'Content-Encoding': 'gzip', 'Location': '/sessions/2', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.2.16.v20160414)'}
INFO:root:Spark session status: starting
INFO:root:Spark session status: starting
INFO:root:Spark session status: starting
INFO:root:Spark session status: starti

NameError: name 'args_q' is not defined